# Exploración Profunda de Emociones Textuales

Hoy en día las redes sociales son una plataforma poderosa para la autoexpresíon y, al mismo tiempo, un lugar donde se puede detectar el estado emocional de las personas. Buscamos no solo identificar la naturaleza emocional de los textos sino también proporcionar una herramienta analítica que pueda ser  útil para intervenir de manera oportuna en situaciones de crisis emocional o psicológica. Esto podría ser de gran ayuda para organizaciones dedicadas al bienestar mental, empresas de redes sociales, y agencias gubernamentales interesadas en monitorear y mejorar la salud emocional de la población.

Tendremos un como entrada un conjunto de datos obtenidos de la plataforma Kaggle. Este dataset cuenta con 1.6 millones de tweets, donde estos se encuntran claseificados en 0 = negativo, 2 = neutral y 4 = positivo. También podemos encontrar ids, date, flag, user y text. Con esto podremos clasificar nustro texto en positivo y negativo.

**ADQUISICIÓN DE LOS DATOS**

El conjunto de datos a utilizar es "Sentiment140 dataset with 1.6 million tweets" obtenido de https://www.kaggle.com/datasets/kazanova/sentiment140/data. Una vez que se descargue, se debe cambiar el nombre del archivo a 'train.csv'.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



**ADQUISICIÓN DE LOS DATOS**

In [41]:
#Descargar el dataset desde https://www.kaggle.com/datasets/kazanova/sentiment140/data
#Una vez que se descargue, se debe cambiar el nombre del archivo a train.csv

#Cargamos el dataset
dataset = pd.read_csv('train.csv', encoding='ISO-8859-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])

#Mostramos las primeras filas del dataset
print(dataset.head(20), '\n')

    target         ids                          date      flag  \
0        0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1        0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2        0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3        0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4        0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
5        0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY   
6        0  1467811592  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
7        0  1467811594  Mon Apr 06 22:20:03 PDT 2009  NO_QUERY   
8        0  1467811795  Mon Apr 06 22:20:05 PDT 2009  NO_QUERY   
9        0  1467812025  Mon Apr 06 22:20:09 PDT 2009  NO_QUERY   
10       0  1467812416  Mon Apr 06 22:20:16 PDT 2009  NO_QUERY   
11       0  1467812579  Mon Apr 06 22:20:17 PDT 2009  NO_QUERY   
12       0  1467812723  Mon Apr 06 22:20:19 PDT 2009  NO_QUERY   
13       0  1467812771  Mon Apr 06 22:20:19 PDT 2009  NO_QUERY   
14       0

**PROCESAMIENTO DE LOS DATOS Y EXTRACCIÓN DE LAS CARACTERISTICAS**

In [42]:
dataset.columns #Mostramos las columnas del dataset

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [43]:
dataset['user'].value_counts().head(10) #Mostramos los usuarios que más tweets han hecho

user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
mcraddictal        276
tsarnick           248
what_bugs_u        246
Karen230683        238
DarkPiano          236
Name: count, dtype: int64

In [44]:
# Función para limpiar el texto

def limpiar_texto(texto):
    texto = re.sub(r'@[A-Za-z0-9]+', '', texto)  # Eliminar menciones
    texto = re.sub(r'https?://[A-Za-z0-9./]+', '', texto)  # Eliminar enlaces
    texto = re.sub(r'#', '', texto)  # Eliminar hashtags
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar signos de puntuación
    texto = texto.lower()  # Convertir a minúsculas
    return texto

In [45]:
# Aplicar la función de limpieza al texto
dataset['text_clean'] = dataset['text'].apply(limpiar_texto)

# Eliminar columnas innecesarias
dataset = dataset[['target', 'text_clean']]

# Convertir 4 (positivo) a 1 y 0 (negativo) a 0
dataset['target'] = dataset['target'].apply(lambda x: 1 if x == 4 else 0)

# Mostrar las primeras filas del dataset limpio
print(dataset['text_clean'].head(20), '\n')

#eliminar columna de target

0        awww thats a bummer  you shoulda got david ...
1     is upset that he cant update his facebook by t...
2      i dived many times for the ball managed to sa...
3       my whole body feels itchy and like its on fire 
4      no its not behaving at all im mad why am i he...
5                                   not the whole crew 
6                                           need a hug 
7      hey  long time no see yes rains a bit only a ...
8                           _k nope they didnt have it 
9                                        que me muera  
10              spring break in plain city its snowing 
11                            i just repierced my ears 
12     i couldnt bear to watch it  and i thought the...
13     it it counts idk why i did either you never t...
14     i wouldve been the first but i didnt have a g...
15     i wish i got to watch it with you i miss you ...
16    hollis death scene will hurt me severely to wa...
17                                 about to file

# Instalar: pip install wordcloud

In [46]:
from wordcloud import WordCloud
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [47]:
# Tomar una muestra del dataset para acelerar el entrenamiento
sample_size = 10000  # Ajustar el tamaño de la muestra según sea necesario
dataset_sample = dataset.sample(n=sample_size, random_state=42)

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(dataset_sample['text_clean'], dataset_sample['target'], test_size=0.3, random_state=50)

# Tokeniza los textos
vocab_size = 10000
embedding_dim = 100
max_length = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

# Construye el modelo de red convolucional
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

/opt/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
